In [14]:
%pylab qt

import numpy as np
import mahotas as mh
import os
import cv2
import json
import itertools
from time import time
from math import radians
from skimage import io, color, img_as_float, img_as_ubyte
from skimage.transform import rescale
from skimage.util import pad
from IPython.parallel import Client
from skimage.measure import label as sklabel
from sklearn.decomposition import PCA, RandomizedPCA
from scipy.cluster.vq import kmeans2
from sklearn.cluster import DBSCAN
from sklearn.svm import SVC
from matplotlib.patches import Rectangle

import helpers.features as fh
import helpers.display as dh

BASE_PATH = os.getcwd()
print("Current base path: {0}".format(BASE_PATH))
DATA_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_cut_2/'
TO_PATH = BASE_PATH + '/Daten/2D/Talus_dorsal_outlined/'
TEST_FILES = json.loads(open(BASE_PATH + '/Daten/2D/testfiles.json').read())

MIN_SIZE = 10
REQUIRED_DIFFERENCE_TO_SPLIT = 0.3
FEATURE_FN = mh.features.pftas

def do_split(image, partition, force):
    rect = partition['rect']
    current_features = partition['features']
    split_y = rect[0] + int((rect[1] - rect[0]) / 2)
    split_x = rect[2] + int((rect[3] - rect[2]) / 2)
    partition_size = (rect[1] - rect[0]) * (rect[3] - rect[2]) / 4
    possible_partitions = (
        {
            'rect': [rect[0], split_y, rect[2], split_x],
            'features': FEATURE_FN(image[rect[0]:split_y, rect[2]:split_x, :]).flatten()
        },
        {
            'rect': [split_y, rect[1], rect[2], split_x],
            'features': FEATURE_FN(image[split_y:rect[1], rect[2]:split_x, :]).flatten()
        },
        {
            'rect': [rect[0], split_y, split_x, rect[3]],
            'features': FEATURE_FN(image[rect[0]:split_y, split_x:rect[3], :]).flatten()
        },
        {
            'rect': [split_y, rect[1], split_x, rect[3]],
            'features': FEATURE_FN(image[split_y:rect[1], split_x:rect[3], :]).flatten()
        }
    )
    
    if force:
        return possible_partitions
    if rect[1] - split_y < MIN_SIZE or split_y-rect[0] < MIN_SIZE:
        return None
    if rect[3] - split_x < MIN_SIZE or split_x-rect[2] < MIN_SIZE:
        return None
    
    combinations = itertools.combinations(possible_partitions, 2)
    distances = [ np.linalg.norm(a['features'] - b['features']) for a, b in combinations ]
    #print(distances)
    #asd = asdf
    if max(distances) > REQUIRED_DIFFERENCE_TO_SPLIT:
        return possible_partitions
    
    return None

def extract_partitions(image):
    partitions = [
        {
            'rect': np.array([ 0, image.shape[0], 0, image.shape[1] ]),
            'features': FEATURE_FN(image).flatten()
        }
    ]
    i = 0
    finished_partitions = []
    
    while len(partitions) != 0:
        partition = partitions.pop(0)
        
        split = do_split(image, partition, i <= 21)
        i += 1
        if split:
            partitions += split
        else:
            finished_partitions.append(partition)
    return finished_partitions

def append_locality(features, partitions, rgb_image):
    locality = np.zeros((features.shape[0], 2))
    print(features.shape[0])
    print(locality.shape)
    for i, p in enumerate(partitions):
        rect = p['rect']
        center_y = (rect[0] + (rect[1] - rect[0]) / 2.0) / rgb_image.shape[0]
        center_x = (rect[2] + (rect[3] - rect[2]) / 2.0) / rgb_image.shape[1]
        
        locality[i, :] = center_y, center_x
       
    return np.concatenate((features, locality), axis=1)

def do_clustering(features):
    #db = DBSCAN(eps=0.3, min_samples=25).fit(features)
    #labels = db.labels_
    centroids, labels = kmeans2(features, 5, iter=25)
    return labels
    
def reduce_feature_complexity(features):
    pca = PCA(n_components=6)
    #pca = RandomizedPCA(n_components=10)
    reduced = pca.fit_transform(features)
    return reduced

def get_bone_clusters(partitions):
    partition_centers = np.zeros((len(partitions), 2))
    partition_labels = np.array([ p['label'] for p in partitions ])
    labels = np.array(range(partition_labels.min(), partition_labels.max()+1))
    centroids = np.zeros((labels.shape[0], 2))
    print(labels)
    
    for l in labels:
        indices = np.argwhere(partition_labels == l)
        num_indices = float(indices.shape[0])
        for i in indices:
            rect = partitions[i]['rect']
            partition_centers[i, :] = [rect[0] + (rect[1] - rect[0]) / 2.0, rect[2] + (rect[3] - rect[2]) / 2.0]
            centroids[l, :] = centroids[l, :] + partition_centers[i, :] / num_indices
        
    mean_distance_from_centroids = np.zeros((labels.shape[0]))
    for l in labels:
        centers = partition_centers[partition_labels == l, :]
        num_partitions_in_cluster = centers.shape[0]
        to_centroid = centers - np.tile(centroids[l, :], (num_partitions_in_cluster, 1))
        mean_distance_from_centroids[l] = np.mean(np.linalg.norm(to_centroid, axis=1))
        
    print(mean_distance_from_centroids)
    dist_centroids, dist_labels = kmeans2(mean_distance_from_centroids, 2, iter=25)
    bone_clusters = labels[dist_labels == np.argmin(dist_centroids)]
    
    return bone_clusters

Populating the interactive namespace from numpy and matplotlib
Current base path: C:\Users\Stefan\Dropbox\Masterarbeit


`%matplotlib` prevents importing * from pylab and numpy


In [15]:
file = TEST_FILES[3]
rgb_image = io.imread(DATA_PATH + file['filename'])

partitions = extract_partitions(rgb_image)
print(len(partitions))

2896


In [9]:
fig, axes = plt.subplots()
axes.imshow(rgb_image)
for partition in partitions:
    rect = partition['rect']
    axes.add_patch(Rectangle((rect[2], rect[0]), rect[3] - rect[2], rect[1]-rect[0], fill=None))

In [16]:
reduced = reduce_feature_complexity(np.array([ p['features'] for p in partitions ]))
#reduced = append_locality(reduced, partitions, rgb_image)

reduced_length = reduced.shape[1]
reduced_image = np.zeros((rgb_image.shape[0], rgb_image.shape[1], reduced_length))
for i, p in enumerate(partitions):
    rect = p['rect']
    features = reduced[i, :]
    reduced_image[rect[0]:rect[1], rect[2]:rect[3], :] = features
reduced_image = reduced_image.reshape((reduced_image.shape[0] * reduced_image.shape[1], reduced_length))

In [53]:
dh.features(plt, rgb_image, reduced_image)

In [17]:
start_time = time()
labels = do_clustering(reduced)
for i, l in enumerate(labels):
    partitions[i]['label'] = l
print('do_clustering: {0}s'.format(time() - start_time))

do_clustering: 0.0s


In [20]:
label_image = np.zeros((rgb_image.shape[0], rgb_image.shape[1]))
for p in partitions:
    rect = p['rect']
    label_image[rect[0]:rect[1], rect[2]:rect[3]] = p['label']
dh.clusters(plt, rgb_image, label_image)

In [18]:
bone_clusters = get_bone_clusters(partitions)

[0 1 2 3 4]
[ 422.98861412  589.31007708  411.2609151   463.85217438  577.1389891 ]


In [19]:
label_image = np.zeros((rgb_image.shape[0], rgb_image.shape[1]))
for p in partitions:
    rect = p['rect']
    if p['label'] in bone_clusters:
        label_image[rect[0]:rect[1], rect[2]:rect[3]] = 1
label_image = sklabel(label_image, 4)
dh.clusters(plt, rgb_image, label_image)